# PyTorch Initialization Methods

在這個 Notebook 裡，我們會示範常見的 PyTorch 參數初始化方法與如何使用它們。所有這些方法都在 `torch.nn.init` 模組中提供，且在初始化參數時會自動進入 `no_grad()` 模式，也就是不會被 Autograd 追蹤梯度。

## Outline
1. `calculate_gain` 取得對應非線性函式的建議增益值（gain）
2. 常見填值初始化：
   - `uniform_` (均勻分布)
   - `normal_` (常態分布)
   - `constant_` (常數填充)
   - `ones_` / `zeros_` (填充 1 或 0)
   - `eye_` (單位矩陣)
   - `dirac_` (Dirac delta)
3. Xavier/Glorot 初始化：
   - `xavier_uniform_`
   - `xavier_normal_`
4. Kaiming/He 初始化：
   - `kaiming_uniform_`
   - `kaiming_normal_`
5. 其他初始化：
   - `trunc_normal_` (截斷常態分布)
   - `orthogonal_` (正交初始化)
   - `sparse_` (稀疏矩陣初始化)


## 1. `calculate_gain`

PyTorch 提供了一個方便的函式 `calculate_gain(nonlinearity, param=None)` 來取得對應非線性函式的建議增益值 (gain)，常搭配 Xavier/He 初始化時使用。

| nonlinearity   | gain (常見設定)                |
|----------------|--------------------------------|
| linear/identity| 1                              |
| conv           | 1                              |
| sigmoid        | 1                              |
| tanh           | $\frac{5}{3}$                 |
| relu           | $\sqrt{2}$                    |
| leaky_relu     | $\sqrt{2/(1+\alpha^2)}$       |
| selu           | 3\~4 (實務可能改用 `linear`) |

舉例來說，在使用 `leaky_relu` 時，可將 `param` 設為 negative_slope，用來計算正確的增益值。

In [1]:
import torch
import torch.nn as nn

gain_relu = nn.init.calculate_gain('relu')
gain_leaky = nn.init.calculate_gain('leaky_relu', param=0.2)

print('Gain for ReLU:', gain_relu)
print('Gain for Leaky ReLU with negative_slope=0.2:', gain_leaky)

Gain for ReLU: 1.4142135623730951
Gain for Leaky ReLU with negative_slope=0.2: 1.3867504905630728


## 2. 常見填值初始化

以下展示幾種最簡單也最常用的初始化方式：

### 2.1 `uniform_`
- 以均勻分布 `U(a, b)` 填充 Tensor
<img src='./images/Uniform.png' >

### 2.2 `normal_`
- 以常態分布 `N(mean, std^2)` 填充 Tensor
<img src='./images/Normal.png' width="600" >

### 2.3 `constant_`
- 以固定常數 `val` 填充 Tensor

### 2.4 `ones_` / `zeros_`
- 分別以 1 或 0 填充 Tensor

### 2.5 `eye_`
- 只適用於 **2D** Tensor，填充成單位矩陣

### 2.6 `dirac_`
- 一般用於卷積核，保留盡可能多的輸入通道特徵（相當於"維持" identity）

In [2]:
# 建立幾個空的 Tensor 來示範不同初始化
w_uniform = torch.empty(3, 5)
w_normal = torch.empty(3, 5)
w_const = torch.empty(3, 5)
w_ones = torch.empty(3, 5)
w_zeros = torch.empty(3, 5)
w_eye = torch.empty(3, 3)

# 初始化示範
nn.init.uniform_(w_uniform, a=0.0, b=1.0)
nn.init.normal_(w_normal, mean=0.0, std=1.0)
nn.init.constant_(w_const, 0.3)
nn.init.ones_(w_ones)
nn.init.zeros_(w_zeros)
nn.init.eye_(w_eye)

print('Uniform distribution [0,1):\n', w_uniform)
print('\nNormal distribution mean=0, std=1:\n', w_normal)
print('\nConstant = 0.3:\n', w_const)
print('\nOnes:\n', w_ones)
print('\nZeros:\n', w_zeros)
print('\nIdentity (eye):\n', w_eye)

Uniform distribution [0,1):
 tensor([[0.2671, 0.4670, 0.3027, 0.2787, 0.2288],
        [0.5017, 0.9847, 0.0516, 0.2456, 0.4050],
        [0.1216, 0.8835, 0.2338, 0.3601, 0.3058]])

Normal distribution mean=0, std=1:
 tensor([[-0.0028,  0.2040, -0.1246, -1.0559, -1.4040],
        [ 0.1987,  1.4315,  1.4492, -0.3885, -0.4568],
        [ 0.8681,  1.0814,  0.8029, -0.1495, -0.1098]])

Constant = 0.3:
 tensor([[0.3000, 0.3000, 0.3000, 0.3000, 0.3000],
        [0.3000, 0.3000, 0.3000, 0.3000, 0.3000],
        [0.3000, 0.3000, 0.3000, 0.3000, 0.3000]])

Ones:
 tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

Zeros:
 tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Identity (eye):
 tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])


## 3. Xavier / Glorot 初始化

這兩種方法常見於深度網路的初始權重設置，以避免梯度在前向/反向傳播中出現大量的增大或縮小。

- `xavier_uniform_`: 依照
  $
  U(-a, a)\quad \text{where}\quad a = \text{gain} \times \sqrt{\frac{6}{fan\_in + fan\_out}}
  $

- `xavier_normal_`: 依照
  $
  N(0, \sigma^2)\quad \text{where}\quad \sigma = \text{gain} \times \sqrt{\frac{2}{fan\_in + fan\_out}}
  $

其中：
- `fan_in`：輸入特徵的數量（輸入層神經元數）
- `fan_out`：輸出特徵的數量（輸出層神經元數）
- `gain`：根據非線性函數計算得到的增益值

這種初始化方法特別適合使用 sigmoid 或 tanh 等激活函數的網路層。

**注意**：PyTorch 的 `fan_in`, `fan_out` 是以權重矩陣在"轉置"的情況下計算。若你的權重維度是 `[fan_in, fan_out]`，請傳入 `w.T` 來取得正確的初始化。

In [3]:
# 建立一個維度為 (3, 5) 的 Tensor，對應 linear layer: fan_in=5, fan_out=3
w_xavier_uniform = torch.empty(3, 5)
w_xavier_normal = torch.empty(3, 5)

# 計算適用 ReLU 的 gain
gain_relu = nn.init.calculate_gain('relu')

# 使用 Xavier 初始化
nn.init.xavier_uniform_(w_xavier_uniform, gain=gain_relu) #Attention 是用這種方式初始化
nn.init.xavier_normal_(w_xavier_normal, gain=gain_relu)

print('Xavier Uniform with ReLU gain:\n', w_xavier_uniform)
print('\nXavier Normal with ReLU gain:\n', w_xavier_normal)

Xavier Uniform with ReLU gain:
 tensor([[-1.0062, -0.5872, -0.1699, -0.4298,  0.1111],
        [-0.8025, -0.6520,  0.5045,  0.7045, -0.2590],
        [ 0.6635, -0.4752,  0.8562, -0.8945, -0.4691]])

Xavier Normal with ReLU gain:
 tensor([[ 0.4047,  0.5327, -0.2970,  0.1878, -0.0796],
        [ 0.4851, -0.5323,  0.2778, -0.7407, -0.4531],
        [ 0.3065,  0.2383,  0.7162,  0.9930,  0.1819]])


## 4. Kaiming / He 初始化

Kaiming 初始化（也稱 He 初始化）常用於 ReLU、Leaky ReLU 等激活函式，能在深度網路初始訓練時維持較穩定的梯度。

- `kaiming_uniform_`: 依照
  $
  U(-bound, bound)\quad \text{where}\quad bound = \text{gain} \times \sqrt{\frac{3}{fan\_mode}}
  $
  
- `kaiming_normal_`: 依照
  $
  N(0, \sigma^2)\quad \text{where}\quad \sigma = \text{gain} \times \sqrt{\frac{1}{fan\_mode}}
  $
  
其中 `fan_mode` 可能是 `fan_in` 或 `fan_out`，對應到是否要在前向傳播或反向傳播中維持方差。默認為 `fan_in`。

**注意**：同樣需要注意 Tensor 維度的轉置問題。

In [4]:
# 建立一個維度為 (3, 5) 的 Tensor，對應 linear layer: fan_in=5, fan_out=3
w_kaiming_uniform = torch.empty(3, 5)
w_kaiming_normal = torch.empty(3, 5)

# 以 leaky_relu 為例，給定 a=0.2
nn.init.kaiming_uniform_(w_kaiming_uniform, a=0.2, mode='fan_in', nonlinearity='leaky_relu')
nn.init.kaiming_normal_(w_kaiming_normal, a=0.2, mode='fan_in', nonlinearity='leaky_relu')

print('Kaiming Uniform (fan_in, a=0.2):\n', w_kaiming_uniform)
print('\nKaiming Normal (fan_in, a=0.2):\n', w_kaiming_normal)

Kaiming Uniform (fan_in, a=0.2):
 tensor([[-0.3712,  0.9724,  0.1284, -0.1359, -1.0139],
        [ 1.0590, -0.0249,  0.2663, -0.5897,  1.0453],
        [-0.9065,  0.1635,  0.2030, -0.4129, -0.3626]])

Kaiming Normal (fan_in, a=0.2):
 tensor([[ 0.2862,  0.5285,  0.6730, -0.2073,  0.0544],
        [-0.0753,  0.6546,  0.2149, -0.3965, -0.5786],
        [ 0.6175, -0.6493,  1.1136, -0.0701,  0.3327]])


## 5. 其他初始化

### 5.1 `trunc_normal_`
- 從常態分布截取至 \([a, b]\) 範圍內，超過範圍的部分重新抽樣，適合一些比較需要限制初始範圍的情況。

### 5.2 `orthogonal_`
- 將最後兩個維度展平後，生成一個近似正交矩陣再填入其中。

### 5.3 `sparse_`
- 生成一個稀疏矩陣（只有少部分元素非 0），非 0 元素從常態分布取樣。

In [5]:
# trunc_normal_
w_trunc_normal = torch.empty(3, 5)
nn.init.trunc_normal_(w_trunc_normal, mean=0.0, std=1.0, a=-2.0, b=2.0)

# orthogonal_
w_orthogonal = torch.empty(5, 5)
nn.init.orthogonal_(w_orthogonal, gain=1.0)

# sparse_
w_sparse = torch.empty(3, 5)
nn.init.sparse_(w_sparse, sparsity=0.8, std=0.01)

print('Trunc Normal (a=-2, b=2):\n', w_trunc_normal)
print('\nOrthogonal (5x5):\n', w_orthogonal)
print('\nSparse (3x5, 80% zero in each column):\n', w_sparse)

Trunc Normal (a=-2, b=2):
 tensor([[ 0.4219,  1.2257, -0.2164,  0.3166, -0.2780],
        [ 0.0491, -1.7142, -0.3582, -0.7454,  0.2659],
        [-0.4679,  0.1626, -0.0602,  0.7535,  0.9937]])

Orthogonal (5x5):
 tensor([[-0.3369, -0.7080, -0.2599,  0.5154,  0.2281],
        [ 0.5478,  0.3734, -0.4644,  0.5632,  0.1660],
        [ 0.1015, -0.2155, -0.7819, -0.5540, -0.1582],
        [ 0.6933, -0.4781,  0.3014, -0.2231,  0.3876],
        [ 0.3090, -0.2905,  0.1206,  0.2459, -0.8632]])

Sparse (3x5, 80% zero in each column):
 tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


## 結論
以上是 PyTorch 初始化函式的一些整理與示範。實務上較常用的初始化方式包括：
- **Xavier / Glorot**：對於 sigmoid、tanh 等激活常有效
- **Kaiming / He**：對於 ReLU、Leaky ReLU 效果良好

也可視需求自行組合 `calculate_gain` 來計算合適的 gain。

若對某些模組或層有特別的初始化需求，也可以透過手動的方式在定義 `nn.Module` 後呼叫相應的初始化函式或自訂函式。

---
### 參考
- [PyTorch 官方文件: torch.nn.init](https://pytorch.org/docs/stable/nn.init.html)
- He et al., [*Delving deep into rectifiers: Surpassing human-level performance on ImageNet classification* (2015)](https://arxiv.org/abs/1502.01852)
- Glorot & Bengio, [*Understanding the difficulty of training deep feedforward neural networks* (2010)](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
